In [1]:
import numpy as np
import pandas as pd
import glob 
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import ast

Remove stopwords?
Combine all for points?
What if word not exist?

### Getting Word Embeddings, Attention Vectors, and Matching Vectors using GloVe

In [2]:
embedding_index = {}

f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))

found word vecs:  400000


In [3]:
embedding_index['unk']

array([ 0.027166 , -0.1762   , -0.19623  ,  0.33527  ,  0.062392 ,
       -0.29748  ,  0.46416  , -0.13271  ,  0.38312  , -0.37872  ,
        0.53709  , -0.071442 , -0.15837  ,  0.049163 , -0.2225   ,
        0.33305  ,  0.11272  , -0.10672  ,  0.48584  , -0.19743  ,
       -0.8179   , -0.43915  , -0.68904  , -0.23316  ,  0.082357 ,
        0.49117  , -0.08154  ,  0.10273  ,  0.39784  ,  0.68379  ,
       -0.072532 , -0.26836  ,  0.52545  , -0.14456  , -0.3563   ,
       -0.19699  ,  0.28968  , -0.71389  ,  0.10056  ,  0.41272  ,
        0.52239  , -0.26027  ,  0.016722 ,  0.54869  ,  0.076753 ,
        0.010193 , -0.11065  ,  0.47543  , -0.20591  ,  0.11673  ,
       -0.096665 ,  0.13824  , -0.16024  , -0.43239  ,  0.23957  ,
        0.21804  ,  0.48378  ,  0.27944  , -0.74636  , -0.60598  ,
       -0.37709  ,  0.28384  , -0.1613   , -0.0017167,  0.35972  ,
        0.091564 , -0.070139 , -0.9614   , -0.25841  ,  0.33928  ,
        0.44946  , -0.1805   ,  0.051543 ,  0.26107  ,  0.1668

In [4]:
main_points =pd.read_csv('DebateStar/Meta Data/metadata_appended_main_points.csv') 
main_points.head()

,id,title,date,for,against,For_Main_Points,against_Main_Points
0,d20191112,Capitalism Is a Blessing,2019-11-12,"['John Mackey', 'Katherine Mangu-Ward']","['Bhaskar Sunkara', 'Richard D. Wolff']",['By promoting market competition and rewardin...,['Capitalism serves the interests of large cor...
1,d20191029,Parenting Is Overrated,2019-10-29,"['Robert Plomin', 'Nancy Segal']","['Paige Harden', 'Ann Pleshette Murphy']","[""We're in the midst of a DNA revolution: Whil...","['While DNA is important, factors like familia..."
2,d20191022,Europe Has Declared War on American Tech Compa...,2019-10-22,"['Roslyn Layton', 'Berin Szóka']","['Marietje Schaake', 'Ramesh Srinivasan']",['European regulators have declared war on Ame...,['Brussels isn’t waging war on Silicon Valley....
3,d20190917,Replace Private Insurance with Medicare for All,2019-09-17,"['Dr. Adam Gaffney', 'Joseph Sanberg']","['Nick Gillespie', 'Sally Pipes']",['The United States government should follow t...,['Individuals should have the freedom to choos...
4,d20190912,Unresolved: Shifting Power in the Middle East,2019-09-12,"['Michael Doran', 'Reuel Marc Gerecht', 'Berna...","['Brett McGurk', 'Barbara Slavin']",NaN,NaN


In [5]:
def get_main_points(fid, main_points):
    """
    Returns main points for both sides given fid and main_points table
    """
    for_points = main_points.loc[main_points['id'] == fid]['For_Main_Points'].item()
    for_points = ast.literal_eval(for_points)
    against_points = main_points.loc[main_points['id'] == fid]['against_Main_Points'].item()
    against_points = ast.literal_eval(against_points)
    return for_points, against_points

In [6]:
punctuations = string.punctuation
def tokenize_point(point):
    """
    Returns a list of lowercased tokens, without punctuations
    """
    tokens = [t for t in word_tokenize(point.lower()) if t not in punctuations]
    return tokens

In [7]:
def word2glove(word, embedding_index):
    """
    Returns GloVe embedding given a word, and zeros if word does not exist in the dictionary
    """
    word = word.lower()
    try:
        word_vec = embedding_index[word]
    except:
        word_vec = embedding_index['unk']
    return word_vec

def point2gloves(point, embedding_index):
    """
    Returns a list of word embeddings given a sentence
    """
    word_vecs = np.array([word2glove(word, embedding_index) for word in tokenize_point(point)])
    return word_vecs
        

In [8]:
def attention_vector(word, embedding_index, title_vecs):
    """
    Returns the attention vector of a word w.r.t. the title (a_k in paper)
    """
    word_vec = word2glove(word, embedding_index)
    #weights = np.dot([word_vec], title_vecs)
    # normalized dot product, then softmax as weights
    weights = np.sum(word_vec * title_vecs, axis=1) / np.linalg.norm(title_vecs, axis=1)
#     print(weights)
#     print(np.exp(weights) / np.sum(np.exp(weights)))
    weights = np.exp(weights) / np.sum(np.exp(weights))
    attention_vec = np.matmul(title_vecs.T, weights.reshape((-1,1))).T[0]
    return attention_vec

In [9]:
# np.concatenate((attention_vector('king', embedding_index, title_vecs), \
#                word2glove('the', embedding_index)), axis = None)

In [10]:
def matching_vector(attention_vec, word, embedding_index):
    """
    Returns the matching vector of a word w.r.t. the title by concatenating the 
    resulting attention vector and word embedding(m_k in paper)
    """
    matching_vec = np.concatenate((attention_vec, word2glove(word, embedding_index)), axis = None)
    return matching_vec

In [11]:
fid = 'd20191112'
title = main_points.loc[main_points['id'] == fid]['title'].iloc[0]
for_points, against_points = get_main_points(fid, main_points)
print('Debate Title:', title)

Debate Title: Capitalism Is a Blessing


In [12]:
word = 'socialism'
title_vecs = point2gloves(title, embedding_index)
attention_vec = attention_vector(word, embedding_index, title_vecs)
matching_vec = matching_vector(attention_vec, word, embedding_index)

In [13]:
print('word:', word)
print('Similarities between attention vector and words in title:', cosine_similarity([attention_vec], title_vecs))

word: socialism
Similarities between attention vector and words in title: [[0.99657255 0.3246895  0.23374872 0.22429445]]


In [14]:
matching_vec.shape

(200,)

In [33]:
def point2mn(point, title, embedding_index):
    #point_vecs = point2gloves(point, embedding_index)
    words = tokenize_point(point)
    title_vecs = point2gloves(title, embedding_index)
    nn_inputs = []
    for word in words:
        attention_vec = attention_vector(word, embedding_index, title_vecs)
        matching_vec = matching_vector(attention_vec, word, embedding_index)
        nn_inputs.append(matching_vec)
    return np.array(nn_inputs)

In [32]:
result = word2mn(point, title, embedding_index)